In [1]:
import pandas as pd
import numpy as np

#### Importing the Postal code dataset and exploring basics of data

In [29]:
df = pd.read_excel(r"C:\Users\The Godfather\Desktop\postalcode_list.xlsx")

In [30]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [31]:
df.shape

(180, 3)

In [44]:
df.describe(include='all')

,Postal code,Borough,Neighborhood
count,180,180,180
unique,180,11,99
top,M3L,Not assigned,Not assigned
freq,1,77,77


In [45]:
df.isnull().sum()

Postal code     0
Borough         0
Neighborhood    0
dtype: int64

#### Dropping (ignoring) rows from the data frame where Borough is "Not assigned"

In [46]:
df.drop(df[df['Borough'] == "Not assigned"].index, inplace = True)# drop rows based on certain column condition
df.reset_index(drop=True, inplace= True) # then reset the index
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


#### Checking if there are anly null values in the df

In [47]:
df.isnull().values.any()

False

#### Replacing / with , as requested in instruction

In [48]:
df["Neighborhood"] = df["Neighborhood"].str.replace("/",",")
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [49]:
df.shape

(103, 3)

In [57]:
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [52]:
df_latlong = pd.read_csv(r"C:\Users\The Godfather\Desktop\Geospatial_Coordinates.csv")
df_latlong.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [53]:
df_latlong.shape

(103, 3)

In [54]:
df_latlong.describe()

,Latitude,Longitude
count,103.000000,103.000000
mean,43.704608,-79.397153
std,0.052463,0.097146
min,43.602414,-79.615819
25%,43.660567,-79.464763
50%,43.696948,-79.388790
75%,43.745320,-79.340923
max,43.836125,-79.160497


In [55]:
df_latlong.isnull().sum()

Postal Code    0
Latitude       0
Longitude      0
dtype: int64

In [63]:
df_latlong.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
df_latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [64]:
df.rename(columns={"Postal code":"PostalCode"}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


#### Merging both dataframes to get result

In [65]:
df_combined = df.merge(df_latlong, on="PostalCode", how='left')
df_combined.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [67]:
df_combined.shape

(103, 5)